In [19]:
import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    res = 12*x*x-96*x+140
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 100
    child_num = 50

    space = OrderedDict((
        ('x', (-2., 150.)),
        ('y', (0., 0.)),
        ('z', (0., 0.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval 131.109232828685, 0.0, 0.0: 193829.08484356228
Eval 24.59905070822403, 0.0, 0.0: 5039.850680959819
Eval 5.219234654377463, 0.0, 0.0: -34.16160229078082
Eval 90.3736544448642, 0.0, 0.0: 89472.89818592972
Eval 84.68644561281236, 0.0, 0.0: 78071.63006755193
Eval 29.587583388748932, 0.0, 0.0: 7804.693084114164
Eval 70.6385771012002, 0.0, 0.0: 53236.399496871236
Eval 22.826212264916627, 0.0, 0.0: 4201.115218924368
Eval 53.99546660916988, 0.0, 0.0: 29942.560177623443
Eval 113.89203039487634, 0.0, 0.0: 144863.1001317011
Eval 8.319265955601704, 0.0, 0.0: 171.8727007426388
Eval -0.7568864817025593, 0.0, 0.0: 219.53562799765464
Eval 132.59571534548945, 0.0, 0.0: 198390.2960626178
Eval 109.68227371700519, 0.0, 0.0: 133972.9157359521
Eval 26.773161476640823, 0.0, 0.0: 6171.402603693892
Eval 119.14565291164999, 0.0, 0.0: 159050.25661340202
Eval 138.62832917574045, 0.0, 0.0: 217445.44419981833
Eval 91.59808551417838, 0.0, 0.0: 92029.09502899171
Eval 69.86603596538384, 0.0, 0.0: 52008.0163255188